In [1]:
import jax
import jax.numpy as jnp

In [2]:
jax.__version__

'0.4.34'

In [3]:
jax.devices()

[CudaDevice(id=0)]

## Using jax.jit as transformation or annotation

In [4]:
def selu(x,
         alpha=1.6732632423543772848170429916717,
         scale=1.0507009873554804934193349852946):
  '''Scaled exponential linear unit activation function.'''
  return scale * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

In [5]:
x = jax.random.normal(jax.random.PRNGKey(42), (1_000_000,))

2024-11-12 10:21:26.244089: W external/xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.3 which is older than the PTX compiler version 12.6.77. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [6]:
selu_jit = jax.jit(selu)

In [7]:
%timeit -n100 selu(x).block_until_ready()

The slowest run took 9.89 times longer than the fastest. This could mean that an intermediate result is being cached.
236 μs ± 309 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit -n100 selu_jit(x).block_until_ready()

The slowest run took 12.94 times longer than the fastest. This could mean that an intermediate result is being cached.
52.7 μs ± 74.8 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
@jax.jit
def selu(x,
         alpha=1.6732632423543772848170429916717,
         scale=1.0507009873554804934193349852946):
  '''Scaled exponential linear unit activation function.'''
  return scale * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

In [10]:
z = selu(x) # warmup

In [11]:
%timeit -n100 selu(x).block_until_ready()

24.1 μs ± 9.51 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Compiling and running on specific hardware

In [12]:
def selu(x,
         alpha=1.6732632423543772848170429916717,
         scale=1.0507009873554804934193349852946):
  '''Scaled exponential linear unit activation function.'''
  return scale * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

In [13]:
selu_jit_cpu = jax.jit(selu, backend='cpu')
selu_jit_gpu = jax.jit(selu, backend='gpu')

In [14]:
%timeit -n100 selu(x).block_until_ready()

135 μs ± 37 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%timeit -n100 selu_jit_cpu(x).block_until_ready()

3.3 ms ± 528 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%timeit -n100 selu_jit_gpu(x).block_until_ready()

The slowest run took 12.79 times longer than the fastest. This could mean that an intermediate result is being cached.
64.9 μs ± 85.4 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
x_cpu = jax.device_put(x, jax.devices('cpu')[0])
x_gpu = jax.device_put(x, jax.devices('gpu')[0])

In [18]:
%timeit -n100 selu(x_cpu).block_until_ready()

491 μs ± 195 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%timeit -n100 selu(x_gpu).block_until_ready()

The slowest run took 4.10 times longer than the fastest. This could mean that an intermediate result is being cached.
215 μs ± 160 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%timeit -n100 selu_jit_cpu(x_cpu).block_until_ready()

The slowest run took 4.75 times longer than the fastest. This could mean that an intermediate result is being cached.
162 μs ± 105 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%timeit -n100 selu_jit_gpu(x_gpu).block_until_ready()

31.1 μs ± 8.64 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%timeit -n100 selu_jit_cpu(x_gpu).block_until_ready()

3.23 ms ± 482 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%timeit -n100 selu_jit_gpu(x_cpu).block_until_ready()

2.14 ms ± 236 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
x_cpu.device

CpuDevice(id=0)

In [25]:
x_gpu.device

CudaDevice(id=0)

In [26]:
selu_jit_gpu

<PjitFunction of <function selu at 0x70d6fc4cc4a0>>

In [27]:
selu_jit_cpu

<PjitFunction of <function selu at 0x70d6fc4cc4a0>>

## Working with function arguments

In [28]:
def dist(order, x, y):
  print("Compiling")
  return jnp.power(jnp.sum(jnp.abs(x-y)**order), 1.0/order)

In [29]:
dist_jit = jax.jit(dist, static_argnums=0)

In [30]:
dist_jit(1, jnp.array([0.0, 0.0]), jnp.array([2.0, 2.0]))

Compiling


Array(4., dtype=float32)

In [31]:
dist_jit(2, jnp.array([0.0, 0.0]), jnp.array([2.0, 2.0]))

Compiling


Array(2.828427, dtype=float32)

In [32]:
dist_jit(1, jnp.array([10.0, 10.0]), jnp.array([2.0, 2.0]))

Array(16., dtype=float32)

In [33]:
from functools import partial

@partial(jax.jit, static_argnums=0)
def dist(order, x, y):
  return jnp.power(jnp.sum(jnp.abs(x-y)**order), 1.0/order)


In [34]:
dist

<PjitFunction of <function dist at 0x70d6fc4ced40>>

In [35]:
dist(2, jnp.array([0.0, 0.0]), jnp.array([2.0, 2.0]))

Array(2.828427, dtype=float32)

In [36]:
def dense_layer(x, w, b, activation_func):
    return activation_func(x*w+b)

In [37]:
x = jnp.array([1.0, 2.0, 3.0])
w = jnp.ones((3,3))
b = jnp.ones(3)

In [38]:
dense_layer_jit = jax.jit(dense_layer)

dense_layer_jit(x, w, b, selu)

TypeError: Error interpreting argument to <function dense_layer at 0x70d6fc340900> as an abstract array. The problematic value is of type <class 'function'> and was passed to the function at path activation_func.
This typically means that a jit-wrapped function was called with a non-array argument, and this argument was not marked as static using the static_argnums or static_argnames parameters of jax.jit.

In [ ]:
dense_layer_jit = jax.jit(dense_layer, static_argnums=3)

In [ ]:
dense_layer_jit(x, w, b, selu)

Array([[2.101402, 3.152103, 4.202804],
       [2.101402, 3.152103, 4.202804],
       [2.101402, 3.152103, 4.202804]], dtype=float32)

## Pure functions again

In [ ]:
global_state = 1

def impure_function(x):
  print(f'Side-effect: printing x={x}')
  y = x*global_state
  return y

In [ ]:
impure_function_jit = jax.jit(impure_function)

In [ ]:
impure_function_jit(10)

Side-effect: printing x=Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=1/0)>


Array(10, dtype=int32, weak_type=True)

In [ ]:
impure_function_jit(10)

Array(10, dtype=int32, weak_type=True)

In [ ]:
impure_function_jit(11)

Array(11, dtype=int32, weak_type=True)

In [ ]:
global_state = 2

In [ ]:
impure_function_jit(10)

Array(10, dtype=int32, weak_type=True)

In [ ]:
impure_function(10)

Side-effect: printing x=10


20

## Jaxpr

Getting jaxpr

In [ ]:
def f1(x, y, z):
  return jnp.sum(x + y * z)

In [ ]:
x = jnp.array([1.0, 1.0, 1.0])
y = jnp.ones((3,3))*2.0
z = jnp.array([2.0, 1.0, 0.0]).T

In [ ]:
jax.make_jaxpr(f1)(x,y,z)

{ lambda ; a:f32[3] b:f32[3,3] c:f32[3]. let
    d:f32[1,3] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 3)] c
    e:f32[3,3] = mul b d
    f:f32[1,3] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 3)] a
    g:f32[3,3] = add f e
    h:f32[] = reduce_sum[axes=(0, 1)] g
  in (h,) }

In [ ]:
f1_jaxpr = jax.make_jaxpr(f1)(x,y,z)

In [ ]:
type(f1_jaxpr)

jax._src.core.ClosedJaxpr

In [ ]:
f1_jaxpr.jaxpr

{ lambda ; a:f32[3] b:f32[3,3] c:f32[3]. let
    d:f32[1,3] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 3)] c
    e:f32[3,3] = mul b d
    f:f32[1,3] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 3)] a
    g:f32[3,3] = add f e
    h:f32[] = reduce_sum[axes=(0, 1)] g
  in (h,) }

In [ ]:
f1_jaxpr.consts

[]

Jaxpr for a function with side-effect

In [ ]:
def f2(x, y):
  print(f'x={x}, y={y}, z={z}')
  return jnp.sum(x + y * z)

In [ ]:
f2_jaxpr = jax.make_jaxpr(f2)(x,y)

x=Traced<ShapedArray(float32[3])>with<DynamicJaxprTrace(level=1/0)>, y=Traced<ShapedArray(float32[3,3])>with<DynamicJaxprTrace(level=1/0)>, z=[2. 1. 0.]


In [ ]:
type(f2_jaxpr.jaxpr)

jax._src.core.Jaxpr

In [ ]:
f2_jaxpr.jaxpr

{ lambda a:f32[3]; b:f32[3] c:f32[3,3]. let
    d:f32[1,3] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 3)] a
    e:f32[3,3] = mul c d
    f:f32[1,3] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 3)] b
    g:f32[3,3] = add f e
    h:f32[] = reduce_sum[axes=(0, 1)] g
  in (h,) }

In [ ]:
f2_jaxpr.jaxpr.constvars

[Var(id=125577265637824):float32[3]]

In [ ]:
f2_jaxpr.jaxpr.invars

[Var(id=125577265637888):float32[3], Var(id=125577265635776):float32[3,3]]

In [ ]:
f2_jaxpr.jaxpr.outvars

[Var(id=125577265639488):float32[]]

In [ ]:
f2_jaxpr.jaxpr.eqns

[a:f32[1,3] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 3)] b,
 a:f32[3,3] = mul b c,
 a:f32[1,3] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 3)] b,
 a:f32[3,3] = add b c,
 a:f32[] = reduce_sum[axes=(0, 1)] b]

In [ ]:
f2_jaxpr.jaxpr.effects

set()

In [ ]:
type(f2_jaxpr.consts)

list

In [ ]:
f2_jaxpr.consts

[Array([2., 1., 0.], dtype=float32)]

In [ ]:
type(jax.make_jaxpr(f2))

function

In [ ]:
jax.grad(f2)(x,y)

x=Traced<ConcreteArray([1. 1. 1.], dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = Array([1., 1., 1.], dtype=float32)
  tangent = Traced<ShapedArray(float32[3])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[3]), None)
    recipe = LambdaBinding(), y=[[2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]], z=[2. 1. 0.]


Array([3., 3., 3.], dtype=float32)

Tracing with control structures

In [ ]:
def f3(x):
  y = x
  for i in range(5):
    y += i
  return y

In [ ]:
jax.make_jaxpr(f3)(0)

{ lambda ; a:i32[]. let
    b:i32[] = add a 0
    c:i32[] = add b 1
    d:i32[] = add c 2
    e:i32[] = add d 3
    f:i32[] = add e 4
  in (f,) }

In [ ]:
jax.jit(f3)(0)

Array(10, dtype=int32, weak_type=True)

In [ ]:
def f4(x):
  y = 0
  for i in range(x.shape[0]):
    y += x[i]
  return y

In [ ]:
jax.make_jaxpr(f4)(jnp.array([1.0, 2.0, 3.0]))

{ lambda ; a:f32[3]. let
    b:f32[1] = slice[limit_indices=(1,) start_indices=(0,) strides=None] a
    c:f32[] = squeeze[dimensions=(0,)] b
    d:f32[] = add 0.0 c
    e:f32[1] = slice[limit_indices=(2,) start_indices=(1,) strides=None] a
    f:f32[] = squeeze[dimensions=(0,)] e
    g:f32[] = add d f
    h:f32[1] = slice[limit_indices=(3,) start_indices=(2,) strides=None] a
    i:f32[] = squeeze[dimensions=(0,)] h
    j:f32[] = add g i
  in (j,) }

In [ ]:
jax.jit(f4)(jnp.array([1.0, 2.0, 3.0]))

Array(6., dtype=float32)

Dependance on a parameter value

In [ ]:
def f5(x):
  y = 0
  for i in range(x):
    y += i
  return y

In [ ]:
f5(5)

10

In [ ]:
jax.make_jaxpr(f5)(5)

TracerIntegerConversionError: The __index__() method was called on traced array with shape int32[]
The error occurred while tracing the function f5 at /tmp/ipykernel_6992/4135095833.py:1 for jit. This concrete value was not available in Python because it depends on the value of the argument x.
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerIntegerConversionError

In [ ]:
jax.jit(f5)(5)

TracerIntegerConversionError: The __index__() method was called on traced array with shape int32[]
The error occurred while tracing the function f5 at /tmp/ipykernel_6992/4135095833.py:1 for jit. This concrete value was not available in Python because it depends on the value of the argument x.
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerIntegerConversionError

In [ ]:
def relu(x):
  #print(x)
  if x > 0:
    return x
  return 0.0

In [ ]:
relu(10.0)

10.0

In [ ]:
jax.make_jaxpr(relu)(10.0)

TracerBoolConversionError: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function relu at /tmp/ipykernel_6992/1355355841.py:1 for jit. This concrete value was not available in Python because it depends on the value of the argument x.
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerBoolConversionError

Using static parameters to overcome this

In [ ]:
jax.make_jaxpr(f5, static_argnums=0)(5)

{ lambda ; . let  in (10,) }

In [ ]:
jax.jit(f5, static_argnums=0)(5)

Array(10, dtype=int32, weak_type=True)

In [ ]:
jax.make_jaxpr(relu, static_argnums=0)(12.3)

{ lambda ; . let  in (12.3,) }

In [ ]:
jax.jit(relu, static_argnums=0)(12.3)

Array(12.3, dtype=float32, weak_type=True)

Rewriting with structured control flow primitives

In [39]:
def f5(x):
  return jax.lax.fori_loop(0, x, lambda i,v: v+i, 0)

In [40]:
f5(5)

Array(10, dtype=int32, weak_type=True)

In [41]:
jax.make_jaxpr(f5)(5)

{ lambda ; a:i32[]. let
    _:i32[] _:i32[] b:i32[] = while[
      body_jaxpr={ lambda ; c:i32[] d:i32[] e:i32[]. let
          f:i32[] = add c 1
          g:i32[] = add e c
        in (f, d, g) }
      body_nconsts=0
      cond_jaxpr={ lambda ; h:i32[] i:i32[] j:i32[]. let
          k:bool[] = lt h i
        in (k,) }
      cond_nconsts=0
    ] 0 a 0
  in (b,) }

In [42]:
jax.jit(f5)(5)

Array(10, dtype=int32, weak_type=True)

In [43]:
def relu(x):
  return jax.lax.cond(x>0, lambda x: x, lambda x: 0.0, x)

In [44]:
relu(12.3)

Array(12.3, dtype=float32, weak_type=True)

In [45]:
relu(-12.3)

Array(0., dtype=float32, weak_type=True)

In [46]:
jax.make_jaxpr(relu)(12.3)

{ lambda ; a:f32[]. let
    b:bool[] = gt a 0.0
    c:i32[] = convert_element_type[new_dtype=int32 weak_type=False] b
    d:f32[] = cond[
      branches=(
        { lambda ; e:f32[]. let  in (0.0,) }
        { lambda ; f:f32[]. let  in (f,) }
      )
    ] c a
  in (d,) }

In [47]:
jax.jit(relu)(12.3)

Array(12.3, dtype=float32, weak_type=True)

## XLA

In [ ]:
def f(x, y, z):
  return jnp.sum(x + y * z)

In [ ]:
x = jnp.array([1.0, 1.0, 1.0])
y = jnp.ones((3,3))*2.0
z = jnp.array([2.0, 1.0, 0.0]).T

In [ ]:
x

Array([1., 1., 1.], dtype=float32)

In [ ]:
y

Array([[2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.]], dtype=float32)

In [ ]:
z

Array([2., 1., 0.], dtype=float32)

In [ ]:
f(x, y, z)

Array(27., dtype=float32)

In [ ]:
f_jitted = jax.jit(f)

In [ ]:
f_lowered = f_jitted.lower(x,y,z)
print(f_lowered.as_text())

module @jit_f attributes {mhlo.num_partitions = 1 : i32, mhlo.num_replicas = 1 : i32} {
  func.func public @main(%arg0: tensor<3xf32> {mhlo.sharding = "{replicated}"}, %arg1: tensor<3x3xf32> {mhlo.sharding = "{replicated}"}, %arg2: tensor<3xf32> {mhlo.sharding = "{replicated}"}) -> (tensor<f32> {jax.result_info = ""}) {
    %0 = stablehlo.broadcast_in_dim %arg2, dims = [1] : (tensor<3xf32>) -> tensor<1x3xf32>
    %1 = stablehlo.broadcast_in_dim %0, dims = [0, 1] : (tensor<1x3xf32>) -> tensor<3x3xf32>
    %2 = stablehlo.multiply %arg1, %1 : tensor<3x3xf32>
    %3 = stablehlo.broadcast_in_dim %arg0, dims = [1] : (tensor<3xf32>) -> tensor<1x3xf32>
    %4 = stablehlo.broadcast_in_dim %3, dims = [0, 1] : (tensor<1x3xf32>) -> tensor<3x3xf32>
    %5 = stablehlo.add %4, %2 : tensor<3x3xf32>
    %6 = stablehlo.constant dense<0.000000e+00> : tensor<f32>
    %7 = stablehlo.reduce(%5 init: %6) applies stablehlo.add across dimensions = [0, 1] : (tensor<3x3xf32>, tensor<f32>) -> tensor<f32>
    retu

In [ ]:
f_compiled = f_lowered.compile()

In [ ]:
f_compiled

In [ ]:
print(f_compiled.as_text())

HloModule jit_f, is_scheduled=true, entry_computation_layout={(f32[3]{0}, f32[3,3]{1,0}, f32[3]{0})->f32[]}, allow_spmd_sharding_propagation_to_output={true}

%region_0.15 (Arg_0.16: f32[], Arg_1.17: f32[]) -> f32[] {
  %Arg_1.17 = f32[] parameter(1)
  %Arg_0.16 = f32[] parameter(0)
  ROOT %add = f32[] add(f32[] %Arg_0.16, f32[] %Arg_1.17), metadata={op_name="jit(f)/jit(main)/reduce_sum[axes=(0, 1)]" source_file="<ipython-input-99-95d48614b981>" source_line=2}
}

%fused_computation (param_0.4: f32[3,3], param_1.4: f32[3], param_2.2: f32[3]) -> f32[] {
  %param_2.2 = f32[3]{0} parameter(2)
  %broadcast.5 = f32[3,3]{1,0} broadcast(f32[3]{0} %param_2.2), dimensions={1}, metadata={op_name="jit(f)/jit(main)/add" source_file="<ipython-input-99-95d48614b981>" source_line=2}
  %param_0.4 = f32[3,3]{1,0} parameter(0)
  %param_1.4 = f32[3]{0} parameter(1)
  %broadcast.4 = f32[3,3]{1,0} broadcast(f32[3]{0} %param_1.4), dimensions={1}, metadata={op_name="jit(f)/jit(main)/mul" source_file="<ipython

MLIR

https://github.com/tensorflow/tensorflow/tree/master/tensorflow/compiler/xla/mlir

https://github.com/tensorflow/tensorflow/tree/master/tensorflow/compiler/xla/mlir_hlo



## JIT Limitations

Long representation

In [ ]:
def cumulative_sum(x):
  acc = 0.0
  y = []
  for i in range(x.shape[0]):
    acc += x[i]
    y.append(acc)
  return y

In [ ]:
cumulative_sum(jnp.array([1.0, 1.0, 5.0, 2.0]))

[Array(1., dtype=float32),
 Array(2., dtype=float32),
 Array(7., dtype=float32),
 Array(9., dtype=float32)]

In [ ]:
j = jax.make_jaxpr(cumulative_sum)(jnp.ones(10000))

In [ ]:
len(j.jaxpr.eqns)

30000

In [ ]:
j

{ lambda ; a:f32[10000]. let
    b:f32[1] = slice[limit_indices=(1,) start_indices=(0,) strides=None] a
    c:f32[] = squeeze[dimensions=(0,)] b
    d:f32[] = add 0.0 c
    e:f32[1] = slice[limit_indices=(2,) start_indices=(1,) strides=None] a
    f:f32[] = squeeze[dimensions=(0,)] e
    g:f32[] = add d f
    h:f32[1] = slice[limit_indices=(3,) start_indices=(2,) strides=None] a
    i:f32[] = squeeze[dimensions=(0,)] h
    j:f32[] = add g i
    k:f32[1] = slice[limit_indices=(4,) start_indices=(3,) strides=None] a
    l:f32[] = squeeze[dimensions=(0,)] k
    m:f32[] = add j l
    n:f32[1] = slice[limit_indices=(5,) start_indices=(4,) strides=None] a
    o:f32[] = squeeze[dimensions=(0,)] n
    p:f32[] = add m o
    q:f32[1] = slice[limit_indices=(6,) start_indices=(5,) strides=None] a
    r:f32[] = squeeze[dimensions=(0,)] q
    s:f32[] = add p r
    t:f32[1] = slice[limit_indices=(7,) start_indices=(6,) strides=None] a
    u:f32[] = squeeze[dimensions=(0,)] t
    v:f32[] = add s u
   

In [ ]:
%time cs = jax.jit(cumulative_sum)(jnp.ones(10000))

CPU times: user 9min 18s, sys: 3.95 s, total: 9min 22s
Wall time: 9min 20s


In [ ]:
def cumulative_sum_fast(x):
  result, array = jax.lax.scan(lambda carry, elem: (carry+elem, carry+elem), 0.0, x)
  return array

In [ ]:
cumulative_sum_fast(jnp.array([1.0, 1.0, 5.0, 2.0]))

Array([1., 2., 7., 9.], dtype=float32)

In [ ]:
j = jax.make_jaxpr(cumulative_sum_fast)(jnp.ones(10000))

In [ ]:
len(j.jaxpr.eqns)

1

In [ ]:
j

{ lambda ; a:f32[10000]. let
    _:f32[] b:f32[10000] = scan[
      jaxpr={ lambda ; c:f32[] d:f32[]. let
          e:f32[] = convert_element_type[new_dtype=float32 weak_type=False] c
          f:f32[] = add e d
          g:f32[] = convert_element_type[new_dtype=float32 weak_type=False] c
          h:f32[] = add g d
        in (f, h) }
      length=10000
      linear=(False, False)
      num_carry=1
      num_consts=0
      reverse=False
      unroll=1
    ] 0.0 a
  in (b,) }

In [ ]:
%time cs = jax.jit(cumulative_sum_fast)(jnp.ones(10000))

CPU times: user 174 ms, sys: 1.01 ms, total: 175 ms
Wall time: 223 ms


Class methods

In [ ]:
class ScaleClass:
  def __init__(self, scale: jnp.array):
    self.scale = scale

  @jax.jit
  def apply(self, x: jnp.array):
    return self.scale * x

In [ ]:
scale_double = ScaleClass(2)

In [ ]:
scale_double.apply(10)

TypeError: ignored

In [ ]:
from functools import partial

class ScaleClass:
  def __init__(self, scale: jnp.array):
    self.scale = scale

  def apply(self, x: jnp.array):
    return _apply_helper(self.scale, x)

@partial(jax.jit, static_argnums=0)
def _apply_helper(scale, x):
  return scale*x

In [ ]:
scale_double = ScaleClass(2)


In [ ]:
scale_double.apply(10)

Array(20, dtype=int32, weak_type=True)

## AOT compilation

In [ ]:
def selu(x,
         alpha=1.6732632423543772848170429916717,
         scale=1.0507009873554804934193349852946):
  '''Scaled exponential linear unit activation function.'''
  print('Function run')
  return scale * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

In [ ]:
selu_jit = jax.jit(selu)

In [ ]:
selu_aot = jax.jit(selu).lower(1.0).compile()

Function run


In [ ]:
selu_jit(17.8)

Array(18.702477, dtype=float32, weak_type=True)

In [ ]:
selu_aot(17.8)

Array(18.702477, dtype=float32, weak_type=True)

In [ ]:
selu_jit(17)

Function run


Array(17.861917, dtype=float32, weak_type=True)

In [ ]:
selu_aot(17)

TypeError: ignored

In [ ]:
selu_jit_batched = jax.vmap(selu_jit)

In [ ]:
selu_aot_batched = jax.vmap(selu_aot)

In [ ]:
selu_jit_batched(jnp.array([42.0, 78.0, -12.3]))

Function run


Array([44.129444 , 81.95468  , -1.7580913], dtype=float32)

In [ ]:
selu_aot_batched(jnp.array([42.0, 78.0, -12.3]))

TypeError: ignored